### Importing Libraries

In [1]:
import matplotlib.pyplot as plt
from glob import glob
import cv2 as cv
import random
import os


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,Flatten,Conv2D
from tensorflow.keras.layers import BatchNormalization,MaxPooling2D,Activation
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG,Image

### Preparing data for training

In [2]:
img_size = 24
batch_size = 32

datagen_train = ImageDataGenerator(rescale = (1./255))
train_generator = datagen_train.flow_from_directory("data/train",
                                                   target_size = (img_size,img_size),
                                                   color_mode = "grayscale",
                                                   batch_size = batch_size,
                                                   class_mode = "categorical",
                                                   shuffle = True)

datagen_validation = ImageDataGenerator(rescale=(1./255))
validation_generator = datagen_validation.flow_from_directory("data/valid",
                                                   target_size = (img_size,img_size),
                                                   color_mode = "grayscale",
                                                   batch_size = batch_size,
                                                   class_mode = "categorical",
                                                   shuffle = True)


Found 3780 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.


### Defining model

In [3]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.25),
    
    Flatten(),
    
    Dense(256, activation='relu'),
    Dropout(0.5),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(2, activation='softmax')
])


### Initialising the model

In [4]:
epochs = 25

steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

### Training the model

In [5]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_generator, validation_data=validation_generator,epochs=20,steps_per_epoch=steps_per_epoch ,validation_steps=validation_steps)


Epoch 1/20


118/118 [==============================] - 11s 56ms/step - loss: 0.4018 - accuracy: 0.8010 - val_loss: 0.6322 - val_accuracy: 0.5521
Epoch 2/20
118/118 [==============================] - 6s 50ms/step - loss: 0.1554 - accuracy: 0.9442 - val_loss: 0.5636 - val_accuracy: 0.6098
Epoch 3/20
118/118 [==============================] - 6s 50ms/step - loss: 0.1282 - accuracy: 0.9544 - val_loss: 0.2693 - val_accuracy: 0.9053
Epoch 4/20
118/118 [==============================] - 6s 50ms/step - loss: 0.1122 - accuracy: 0.9600 - val_loss: 0.1916 - val_accuracy: 0.9233
Epoch 5/20
118/118 [==============================] - 6s 51ms/step - loss: 0.0926 - accuracy: 0.9650 - val_loss: 0.1225 - val_accuracy: 0.9583
Epoch 6/20
118/118 [==============================] - 6s 52ms/step - loss: 0.0870 - accuracy: 0.9685 - val_loss: 0.1026 - val_accuracy: 0.9564
Epoch 7/20
118/118 [==============================] - 6s 50ms/step - loss: 0.0715 - accuracy: 0.9736 - val_loss: 0.0968 - val_accuracy: 0.

### Evaluating Model

In [6]:
model.evaluate(validation_generator)

34/34 [==============================] - 1s 35ms/step - loss: 0.2214 - accuracy: 0.9588


[0.22143414616584778, 0.9588015079498291]

### Saving the model

In [7]:
model.save('eye_model.h5')

C:\Users\Acer\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model_json = model.to_json()
with open("eye_model.json","w") as json_file:
    json_file.write(model_json)